Se crean los dataframes necesarios a partir de df_siniestros

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df_siniestros.csv to df_siniestros.csv


In [ ]:
df_siniestros = pd.read_csv('df_siniestros.csv')

1. Primer KPI - Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior

In [ ]:
#Se crea la variable Semestre. Básicamente la función divide el año en dos#
df_siniestros["Semestre"] = df_siniestros["Mes"].apply(lambda x: 1 if x <= 6 else 2)

In [ ]:
#Se agrupa por año y por semestre para poder sumar la cantidad que corresponde a cada uno#
df_agrupado_semestral = df_siniestros.groupby(["Año","Semestre"]).size().reset_index(name="Homicidios")

In [ ]:
#Se crea una nueva variable llamada homicidios semestre anterior#
df_agrupado_semestral["SemestreAnterior"] = df_agrupado_semestral["Homicidios"].shift(periods=1, fill_value=0)

In [ ]:
#Se calcula la tasa de homicidios cada 10000 habitantes#
df_agrupado_semestral["TasaHomicidios"] = (df_agrupado_semestral["Homicidios"]/3120612) * 100000
df_agrupado_semestral["TasaSemestreAnterior"] = (df_agrupado_semestral["SemestreAnterior"]/3120612) * 100000

In [ ]:
#Se crea una nueva variable llamada diferencia porcentual"
df_agrupado_semestral["Diferencia Porcentual"] = (df_agrupado_semestral["TasaHomicidios"] - df_agrupado_semestral["TasaSemestreAnterior"]) / df_agrupado_semestral["TasaSemestreAnterior"] * 100


In [ ]:
df_agrupado_semestral

,Año,Semestre,Homicidios,SemestreAnterior,TasaHomicidios,TasaSemestreAnterior,Diferencia Porcentual
0,2016,1,64,0,2.050880,0.000000,inf
1,2016,2,78,64,2.499510,2.050880,21.875000
2,2017,1,66,78,2.114970,2.499510,-15.384615
3,2017,2,71,66,2.275195,2.114970,7.575758
4,2018,1,69,71,2.211105,2.275195,-2.816901
5,2018,2,79,69,2.531555,2.211105,14.492754
6,2019,1,57,79,1.826565,2.531555,-27.848101
7,2019,2,47,57,1.506115,1.826565,-17.543860
8,2020,1,30,47,0.961350,1.506115,-36.170213
9,2020,2,50,30,1.602250,0.961350,66.666667


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_agrupado_semestral_csv = "/content/drive/MyDrive/df_kpi1.csv"

df_agrupado_semestral.to_csv(df_agrupado_semestral_csv, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## KPI 2

Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Filtramos y seleccionamos aquellos registros con el motociclista como víctima

In [ ]:
df_kpi2 = df_siniestros[(df_siniestros["Victima"] == "MOTO")]

Agrupamos por Año luego obtenemos la cantidad de homicidios

In [ ]:
df_kpi2 = df_kpi2.groupby(["Año"]).size().reset_index(name="Homicidios")

Se crea la nueva variable Año anterior donde se informa la cantidad de homicidios durante el anterior año

In [ ]:
df_kpi2["Año_Anterior"] = df_kpi2["Homicidios"].shift(periods=1, fill_value=0)

Se obtiene la tasa de homicidios cada 100000 habitantes teniendo en cuenta que la poblacion de Cuidad Autonoma de Buenos Aires al censo 2022

In [ ]:
poblacion_caba = 3120612
df_kpi2["Tasa_Homicidios"] = (df_kpi2["Homicidios"] / poblacion_caba) * 100000
df_kpi2["Tasa_Año_Anterior"] = (df_kpi2["Año_Anterior"] / poblacion_caba) * 100000

Se crea la variable Variacion Porcentual

In [ ]:
df_kpi2["Variacion"] = ((df_kpi2["Tasa_Homicidios"] - df_kpi2["Tasa_Año_Anterior"]) / df_kpi2["Tasa_Año_Anterior"]) * 100
df_kpi2["Variacion"] = df_kpi2["Variacion"].replace([np.inf, -np.inf], np.nan)
df_kpi2

,Año,Homicidios,Año_Anterior,Tasa_Homicidios,Tasa_Año_Anterior,Variacion
0,2016,63,0,2.018835,0.000000,NaN
1,2017,54,63,1.730430,2.018835,-14.285714
2,2018,57,54,1.826565,1.730430,5.555556
3,2019,50,57,1.602250,1.826565,-12.280702
4,2020,27,50,0.865215,1.602250,-46.000000
5,2021,45,27,1.442025,0.865215,66.666667


In [ ]:
df_kpi2_csv = "/content/drive/MyDrive/df_kpi2.csv"
df_kpi2.to_csv(df_kpi2_csv, index=False)